### Merge clinical and meta data

In [21]:
import os
import pandas as pd
from pathlib import Path

df_meta = pd.read_csv('/vol/biomedic3/bglocker/mammo/mammo-net/data/EMBED_OpenData_metadata_reduced.csv')
df_clinical = pd.read_csv('/vol/biomedic3/bglocker/mammo/mammo-net/data/EMBED_OpenData_clinical_reduced.csv')

df_merged = pd.merge(df_meta, df_clinical, on=['empi_anon','acc_anon'])

df_merged['image_path'] = df_merged[['empi_anon', 'anon_dicom_path']].apply(lambda x: os.path.join(str(x[0]), f"{Path(x[1]).stem}.png"), axis=1)

/tmp/ipykernel_139589/469899571.py:5: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta = pd.read_csv('/vol/biomedic3/bglocker/mammo/mammo-net/data/EMBED_OpenData_metadata_reduced.csv')
/tmp/ipykernel_139589/469899571.py:6: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clinical = pd.read_csv('/vol/biomedic3/bglocker/mammo/mammo-net/data/EMBED_OpenData_clinical_reduced.csv')
/tmp/ipykernel_139589/469899571.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_merged['image_path'] = df_merged[['empi_anon', 'anon_dicom_path']].apply(lambda x: os.path.join(str(x[0]), f"{Path(x[1]).stem}.png"), axis=1)


### Split into left/right images, and remove rows where finding and image laterality mismatch

In [22]:
df_left = df_merged[df_merged.ImageLateralityFinal == 'L'].reset_index()
df_right = df_merged[df_merged.ImageLateralityFinal == 'R'].reset_index()
df_left.drop(df_left[df_left.side == 'R'].index, inplace=True)
df_right.drop(df_right[df_right.side == 'L'].index, inplace=True)
df_images = pd.concat([df_left.reset_index(), df_right.reset_index()])
df_images = pd.concat([df_left, df_right], ignore_index=True)
df_images['is_positive'] = 0

In [23]:
df_images.drop(df_images[df_images.asses == 'X'].index, inplace=True)
df_images.drop(df_images[(df_images.asses == 'A') & (df_images.path_severity.isna())].index, inplace=True)

In [24]:
images_path_result = (df_images.ImageLateralityFinal == df_images.bside) | (df_images.bside == 'B')

### Non-negatives

In [ ]:
df_images.loc[images_path_result & (df_images.path_severity.isin([0,1,2,3,4])), 'is_positive'] = 1
df_images.loc[df_images.asses.isin(['B','P','S','M','K']), 'is_positive'] = 1
out_filename = 'data/embed-non-negative.csv'